In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
train

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [ ]:
test

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...
...,...,...
8387,id11749,All this is now the fitter for my purpose.
8388,id10526,I fixed myself on a wide solitude.
8389,id13477,It is easily understood that what might improv...
8390,id13761,"Be this as it may, I now began to feel the ins..."


In [ ]:
sample

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698
...,...,...,...,...
8387,id11749,0.403494,0.287808,0.308698
8388,id10526,0.403494,0.287808,0.308698
8389,id13477,0.403494,0.287808,0.308698
8390,id13761,0.403494,0.287808,0.308698


In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [ ]:
label_enc=preprocessing.LabelEncoder()
y=label_enc.fit_transform(train.author.values)

In [ ]:
y

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(train.text.values,y,stratify=y,random_state=42,test_size=0.1,shuffle=True)
print(X_train.shape)
print(X_val.shape)


(17621,)
(1958,)


#Simple TF-IDF


In [ ]:
tfv=TfidfVectorizer(min_df=3, max_features=None,strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [ ]:
tfv.fit(list(X_train)+list(X_val))
X_train_tfv=tfv.transform(X_train)
X_val_tfv=tfv.transform(X_val)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [ ]:
print(X_train_tfv)

  (0, 11813)	0.2469424385443994
  (0, 10050)	0.31534173991151543
  (0, 7734)	0.2621357570531394
  (0, 6078)	0.21913481148346806
  (0, 5920)	0.2699803359499066
  (0, 5641)	0.28210670387497144
  (0, 3647)	0.33928338965309934
  (0, 3312)	0.2913532806410929
  (0, 2813)	0.3345950093082599
  (0, 2161)	0.3474674493892856
  (0, 1489)	0.3710563608683541
  (1, 14862)	0.3467364483300827
  (1, 11446)	0.23785570257025115
  (1, 9156)	0.3385709613222204
  (1, 8899)	0.49640563902064305
  (1, 8268)	0.4364136618558541
  (1, 4746)	0.3259827441631435
  (1, 72)	0.4067284254948717
  (2, 14071)	0.3240192548729904
  (2, 13042)	0.25915485523649134
  (2, 12014)	0.26382082678856383
  (2, 11546)	0.16409370088652775
  (2, 9656)	0.25408177755421213
  (2, 9422)	0.26141001709560185
  (2, 7995)	0.28804138618156144
  :	:
  (17617, 2312)	0.3765818334354421
  (17618, 14413)	0.41062696740663335
  (17618, 13333)	0.41457868692900307
  (17618, 12513)	0.2857797208393344
  (17618, 8238)	0.3745429502598771
  (17618, 3074)	0.384

In [ ]:
model= LogisticRegression(C=1.0)
model.fit(X_train_tfv,y_train)
pred = model.predict_proba(X_val_tfv)

print ("logloss: %0.3f " % multiclass_logloss(y_val, pred))


logloss: 0.572 


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#CountVectorizer:

In [ ]:
ctv=CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

ctv.fit(list(X_train) + list(X_val))
X_train_ctv =  ctv.transform(X_train)
X_val_ctv = ctv.transform(X_val)

In [ ]:
print(predictions )

[[7.70831266e-01 1.65959928e-02 2.12572741e-01]
 [9.02504834e-01 3.32703987e-02 6.42247673e-02]
 [7.90068438e-01 1.06515383e-01 1.03416178e-01]
 ...
 [3.70004830e-01 1.83420355e-01 4.46574815e-01]
 [1.35184745e-01 6.35598867e-02 8.01255368e-01]
 [1.57127052e-04 9.99824861e-01 1.80115846e-05]]


In [ ]:
model = LogisticRegression()
model.fit(X_train_ctv, y_train)
predictions = model.predict_proba(X_val_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

logloss: 0.527 


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
model = MultinomialNB()

model.fit(X_train_tfv, y_train)
predictions = model.predict_proba(X_val_tfv)

print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

logloss: 0.578 


In [ ]:
model = MultinomialNB()

model.fit(X_train_ctv, y_train)
predictions = model.predict_proba(X_val_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

logloss: 0.485 


In [ ]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(X_train_tfv)
xtrain_svd = svd.transform(X_train_tfv)
xvalid_svd = svd.transform(X_val_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y_train)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

logloss: 0.735 


In [ ]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [ ]:
nb_model = MultinomialNB()

# Create the pipeline
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

# Fit Grid Search Model
model.fit(X_train_tfv, y_train)  # we can use the full data here but im only using xtrain.
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.492
Best parameters set:
	nb__alpha: 0.1


In [ ]:
X_train


array(['Her hair was the brightest living gold, and despite the poverty of her clothing, seemed to set a crown of distinction on her head.',
       '"No," he said, "oh, no a member of my family my niece, and a most accomplished woman."',
       'The magistrate appeared at first perfectly incredulous, but as I continued he became more attentive and interested; I saw him sometimes shudder with horror; at others a lively surprise, unmingled with disbelief, was painted on his countenance.',
       ...,
       'The medical testimony spoke confidently of the virtuous character of the deceased.',
       'When we arrived, after a little rest, he led me over the house and pointed out to me the rooms which my mother had inhabited.',
       'Some were destroyed; the major part escaped by quick and well ordered movements; and danger made them careful.'],
      dtype=object)

#Texte Prep:
-Lowercasing
/Tokenization
/Remove Punct
/Removing Stopwords
/lemmatization


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
X_train

array(['Her hair was the brightest living gold, and despite the poverty of her clothing, seemed to set a crown of distinction on her head.',
       '"No," he said, "oh, no a member of my family my niece, and a most accomplished woman."',
       'The magistrate appeared at first perfectly incredulous, but as I continued he became more attentive and interested; I saw him sometimes shudder with horror; at others a lively surprise, unmingled with disbelief, was painted on his countenance.',
       ...,
       'The medical testimony spoke confidently of the virtuous character of the deceased.',
       'When we arrived, after a little rest, he led me over the house and pointed out to me the rooms which my mother had inhabited.',
       'Some were destroyed; the major part escaped by quick and well ordered movements; and danger made them careful.'],
      dtype=object)

In [ ]:
import re

In [ ]:
def clean_text(text):
    N_text=[]
    for e in text:
        text=e.lower()

        text = re.sub(r"'|\"", '', text)
        tokens=word_tokenize(text)
        tokens = [word for word in tokens if word not in string.punctuation]

        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        cleaned_text = ' '.join(tokens)
        N_text.append(cleaned_text)

    return np.array(N_text)



In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Step 1: Clean the text data
cleaned_text = clean_text(X_train)
word_embeddings_out={}
# Step 2: Tokenization
for e in cleaned_text:
    tokenized_text = word_tokenize(e)

    # Step 3: Train Word2Vec model
    word2vec_model = Word2Vec(sentences=[tokenized_text], vector_size=25, window=5, min_count=1, workers=4)

    # Step 4: Generate word embeddings
    word_embeddings = {}
    for word in tokenized_text:

         if word in word2vec_model.wv:
            word_embeddings[word] = word2vec_model.wv[word]
         else:
            # Handle out-of-vocabulary words
            word_embeddings[word] = None


    all_vectors = np.array(list(word_embeddings.values()))
    mean_vector = np.mean(all_vectors, axis=0)
    word_embeddings_out[e]=mean_vector



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


RuntimeError: you must first build vocabulary before training the model

In [ ]:
x=[x for x in  word_embeddings_out.values()]
x

[array([-0.00369868,  0.0039474 ,  0.00266079, -0.0020914 , -0.00046805,
        -0.00301604,  0.00034976,  0.00556441, -0.01399726, -0.00023942,
        -0.0033946 ,  0.00088955, -0.0065993 ,  0.0074142 ,  0.00221221,
         0.00250948,  0.011141  ,  0.00658585, -0.01080477,  0.0050915 ,
         0.01119539,  0.00082856,  0.00535346, -0.00555803,  0.01178903],
       dtype=float32),
 array([-0.00538024,  0.0002574 ,  0.0087452 ,  0.00408304, -0.00014521,
        -0.00443896,  0.01064391,  0.0166343 , -0.01033013,  0.00170626,
        -0.000248  ,  0.00839054, -0.0134532 ,  0.00420763,  0.00679503,
         0.00752902,  0.00843773, -0.00351545, -0.00680788,  0.00306218,
         0.01375136, -0.00599344,  0.01151867, -0.00251383,  0.01949386],
       dtype=float32),
 array([-0.00254052,  0.00520399,  0.00521328, -0.0058713 , -0.00140421,
        -0.00408382, -0.00116735,  0.0054718 , -0.00942093, -0.0015559 ,
        -0.00206401,  0.00687515, -0.00181484, -0.00080761,  0.00160123,
   

In [ ]:
len(x)

4098

In [ ]:
def multiclass_logloss2(actual, predicted, eps=1e-15):
    """
    Compute the multiclass log loss.

    Parameters:
    actual : array-like, shape (n_samples,)
        Array containing the actual target classes (integers).
    predicted : array-like, shape (n_samples,)
        Array containing the predicted target classes (integers).
    eps : float, optional
        A small value to avoid division by zero.

    Returns:
    float
        Multiclass log loss value.
    """
    # Compute the number of unique classes
    num_classes = len(np.unique(actual))

    # Convert 'actual' and 'predicted' to one-hot encoded arrays
    actual_one_hot = np.eye(num_classes)[actual]
    predicted_one_hot = np.eye(num_classes)[predicted]

    # Clip predicted probabilities to avoid log(0) or log(1)
    predicted_one_hot = np.clip(predicted_one_hot, eps, 1 - eps)

    # Compute log loss
    log_loss = -(actual_one_hot * np.log(predicted_one_hot)).mean()

    return log_loss

In [ ]:
X =  np.array(x)  # Features (encoded sentences)
y =  y_train[:4098]            # Labels (class labels)

# 2. Split Data
X_trai, X_tes, y_trai, y_tes = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Choose a Model
model = LogisticRegression()

# 4. Train the Model
model.fit(X_trai, y_trai)

# 5. Evaluate the Model
y_pred = model.predict(X_tes)



In [ ]:
from sklearn.metrics import r2_score

# Assuming y_true and y_pred are the true and predicted values, respectively
r2 = r2_score(y_tes, y_pred)

print("R^2 Score:", r2)


R^2 Score: -1.0965889269882325
